In [1]:
import pandas as pd
import altair as alt
import numpy as np

### Calculate static gaussian in DataFrame

We use this increment throughout

In [33]:
inc = 0.1

In [3]:
A = 1.0
x0 = 1.0
sig = 1.0

df_gauss = pd.DataFrame({'x':np.arange(-5,10+inc,inc)})
df_gauss['y1'] = df_gauss['x'].apply(lambda x: A*np.exp(-(pow((x-x0),2)/pow(sig,2))))

### Visualize static gaussian with line + area

In [4]:
base = alt.Chart(df_gauss).encode(
    x = 'x',
    y = 'y1'
)

line = base.mark_line(color='black')

area = base.mark_area(color='lightblue')

(line + area).properties(
    height = 150
)

alt.LayerChart(...)

### Area above cutoff in static gaussian

In [5]:
slider = alt.binding_range(min=-5, max=10, step=inc, name='cutoff:')
selector = alt.selection_single(name="SelectorName", fields=['cutoff'],
                                bind=slider, init={'cutoff': 0})

base = alt.Chart(df_gauss).encode(
    x = 'x',
    y = 'y1'
)

line = base.mark_line(color='black')

area = base.mark_area(
    color='lightblue'
).add_selection(
    selector
).transform_filter(
    alt.datum.x >= selector.cutoff
)

bar = alt.Chart(df_gauss).mark_bar().encode(
    y = alt.Y('sum(y1)', 
              scale=alt.Scale(domain=(0, 20)),
              axis = alt.Axis(title='Area of blue region')
             )
).transform_filter(
    alt.datum.x >= selector.cutoff
)

(area + line).properties(height = 150) | bar.properties(height = 150, width=100)

alt.HConcatChart(...)

### Change width, center & cutoff of normalized gaussian

- Creating a new dataframe with an extended X range so areas off the edge of the plot will be calculated properly
- Requires `clip=True` for all `mark_line()`, `mark_area()` specifications to keep data in axis domain

In [34]:
df = pd.DataFrame({'x':np.arange(-10,20+inc,inc)})

In [7]:
center_slider = alt.binding_range(min=-5, max=10, step=inc, name='center:')
center_selector = alt.selection_single(name="CenterName", fields=['center'],
                                bind=center_slider, init={'center': 0})

width_slider = alt.binding_range(min=inc, max=10, step=inc, name='width:')
width_selector = alt.selection_single(name="WidthName", fields=['width'],
                                bind=width_slider, init={'width': 1})

cutoff_slider = alt.binding_range(min=-5, max=10, step=inc, name='cutoff:')
cutoff_selector = alt.selection_single(name="CutoffName", fields=['cutoff'],
                                bind=cutoff_slider, init={'cutoff': 0})

base = alt.Chart(df).transform_calculate(
    ycalc = (1.0/(alt.expr.sqrt(2*alt.expr.PI)*width_selector.width))*alt.expr.exp(-0.5*(pow((alt.datum.x - center_selector.center),2)/pow(width_selector.width,2)))
)

gauss_base = base.encode(
    x = alt.X('x:Q', scale=alt.Scale(domain=[-5,10]), title=''),
    y = alt.Y('ycalc:Q', title='')
)

line = gauss_base.mark_line(
    color='#262626',
    clip=True
).add_selection(
    cutoff_selector, 
    width_selector,
    center_selector
)

area = gauss_base.mark_area(
    color='#C84E0099',
    clip=True
).transform_filter(
    alt.datum.x >= cutoff_selector.cutoff
)

bar = base.mark_bar(size=50, color='#C84E00CC').encode(
    y = alt.Y('sum(ycalc):Q', 
              scale=alt.Scale(domain=(0, 10)),
              axis = alt.Axis(title='Area of orange region')
             )
).transform_filter(
    alt.datum.x >= cutoff_selector.cutoff
)

combo = (area + line).properties(height = 150) | bar.properties(height = 150, width=100)
combo

alt.HConcatChart(...)

In [8]:
combo.save('gaussian_cutoff.html')

### Treatment vs static Control group

In [16]:
xmin = -3
xmax = 5

c_width = 1.0
t_width = 1.0

t_center_slider = alt.binding_range(min=xmin, max=xmax, step=inc, name='Treatment center:')
t_center_selector = alt.selection_single(name="TCenter", fields=['center'],
                                bind=t_center_slider, init={'center': 2})

thresh_slider = alt.binding_range(min=xmin, max=xmax, step=inc, name='Responder threshold:')
thresh_selector = alt.selection_single(name="Threshold", fields=['threshold'],
                                bind=thresh_slider, init={'threshold': 0.5})

base = alt.Chart(df).transform_calculate(
    c_y = (1.0/(alt.expr.sqrt(2*alt.expr.PI)*c_width))*alt.expr.exp(-0.5*(pow((alt.datum.x - 0.0)/c_width,2))),
    t_y = (1.0/(alt.expr.sqrt(2*alt.expr.PI)*t_width))*alt.expr.exp(-0.5*(pow((alt.datum.x - t_center_selector.center)/t_width,2))),
    c_yn = inc*alt.datum.c_y,
    t_yn = inc*alt.datum.t_y
)

c_gauss_base = base.encode(
    x = alt.X('x:Q', scale=alt.Scale(domain=[xmin,xmax]), title=''),
    y = alt.Y('c_y:Q', title='')
)
c_line = c_gauss_base.mark_line(color='#666666',clip=True).add_selection(
    thresh_selector, 
    t_center_selector
)
c_area = c_gauss_base.mark_area(color='#C84E0099',clip=True).transform_filter( alt.datum.x >= thresh_selector.threshold )

t_gauss_base = base.encode(
    x = alt.X('x:Q', scale=alt.Scale(domain=[xmin,xmax]), title=''),
    y = alt.Y('t_y:Q', scale=alt.Scale(domain=[0,0.45]), title='')
)
t_line = t_gauss_base.mark_line(color='#262626',clip=True)
t_area = t_gauss_base.mark_area(color='#33989899',clip=True).transform_filter( alt.datum.x >= thresh_selector.threshold )

c_bar = base.mark_bar(size=30, color='#C84E00AA').encode(
    y = alt.Y('sum(c_yn):Q', scale=alt.Scale(domain=(0, 1)), title='Control area')
).transform_filter( alt.datum.x >= thresh_selector.threshold )
t_bar = base.mark_bar(size=30, color='#339898BB').encode(
    y = alt.Y('sum(t_yn):Q', scale=alt.Scale(domain=(0, 1)), title='Treatment area')
).transform_filter( alt.datum.x >= thresh_selector.threshold )
diff_bar = base.mark_bar(size=30, color='#993399BB').encode(
    y = alt.Y('sum(tc_diff):Q', scale=alt.Scale(domain=(0, 1)), title='Area difference')
).transform_filter( alt.datum.x >= thresh_selector.threshold ).transform_calculate(
    tc_diff = alt.datum.t_yn - alt.datum.c_yn
)

diff_combo = (c_area + t_area + c_line + t_line).properties(height = 200, width=350) | \
        c_bar.properties(height = 200, width=50) | \
        t_bar.properties(height = 200, width=50) | \
        diff_bar.properties(height = 200, width=50)

diff_combo

alt.HConcatChart(...)

In [17]:
diff_combo.save('responder_threshold.html')

### Fancier version with people who got worse

<img src="images/FancierVersion.png" width=500 />

### If conditional to set values to zero in threshold range

**NOTE: Realize now that need to incorporate `inc` in normalization of integration**

In [11]:
fine_inc = 0.01
df_fine = pd.DataFrame({'x':np.arange(-10,20+fine_inc,fine_inc)})

xmin = -3
xmax = 5

c_width = 1.0
t_width = 1.0

t_center_slider = alt.binding_range(min=xmin, max=xmax, step=fine_inc, name='Treatment center:')
t_center_selector = alt.selection_single(name="TCenter", fields=['center'],
                                bind=t_center_slider, init={'center': 2})

thresh_slider = alt.binding_range(min=0, max=xmax, step=fine_inc, name='Responder threshold:')
thresh_selector = alt.selection_single(name="Threshold", fields=['threshold'],
                                bind=thresh_slider, init={'threshold': 0.5})

base = alt.Chart(df_fine).transform_calculate(
    c_y = (1.0/(alt.expr.sqrt(2*alt.expr.PI)*c_width))*alt.expr.exp(-0.5*(pow((alt.datum.x - 0.0)/c_width,2))),
    t_y = (1.0/(alt.expr.sqrt(2*alt.expr.PI)*t_width))*alt.expr.exp(-0.5*(pow((alt.datum.x - t_center_selector.center)/t_width,2)))
)

t_gauss_base = base.encode(
    x = alt.X('x:Q', scale=alt.Scale(domain=[xmin,xmax]), title='')
)
t_line = t_gauss_base.mark_line(color='#262626',clip=True).encode(
    y = alt.Y('t_y:Q', scale=alt.Scale(domain=[0,0.45]), title='')
).add_selection(
    thresh_selector, 
    t_center_selector
)
t_area = t_gauss_base.mark_area(
    color='#33989899',
    clip=True
).encode(
    y = alt.Y('t_y2:Q', scale=alt.Scale(domain=[0,0.45]), title='')
).transform_calculate( 
    t_y2 = alt.expr.if_(alt.expr.inrange(alt.datum.x,[-thresh_selector.threshold,thresh_selector.threshold]),0,alt.datum.t_y)
)

fancy_combo = (t_area + t_line).properties(height = 200, width=350)

fancy_combo

alt.LayerChart(...)

### Separate filtered versions added together

Vega-lite bug where tooltips don't show up in a compound chart:
https://github.com/altair-viz/altair/issues/2009

In [42]:
xmin = -3
xmax = 5

c_width = 1.0
t_width = 1.0

# Selectors (sliders)
t_center_slider = alt.binding_range(min=xmin, max=xmax, step=inc, name='Treatment center:')
t_center_selector = alt.selection_single(name="TCenter", fields=['center'],
                                bind=t_center_slider, init={'center': 2})

thresh_slider = alt.binding_range(min=0, max=xmax, step=inc, name='Responder threshold:')
thresh_selector = alt.selection_single(name="Threshold", fields=['threshold'],
                                bind=thresh_slider, init={'threshold': 0.5})

# Base calculations used in later charts
# Proper integration (approximation) requires not just sum(y) but sum(y*dx)
base = alt.Chart(df).transform_calculate(
    c_y = (1.0/(alt.expr.sqrt(2*alt.expr.PI)*c_width))*alt.expr.exp(-0.5*(pow((alt.datum.x - 0.0)/c_width,2))),
    t_y = (1.0/(alt.expr.sqrt(2*alt.expr.PI)*t_width))*alt.expr.exp(-0.5*(pow((alt.datum.x - t_center_selector.center)/t_width,2))),
    c_yn = inc*alt.datum.c_y,
    t_yn = inc*alt.datum.t_y,
    c_ynPos = alt.expr.if_(alt.datum.x >= thresh_selector.threshold, alt.datum.c_yn, 0),
    c_ynNeg = alt.expr.if_(alt.datum.x <= -thresh_selector.threshold, alt.datum.c_yn, 0),
    t_ynPos = alt.expr.if_(alt.datum.x >= thresh_selector.threshold, alt.datum.t_yn, 0),
    t_ynNeg = alt.expr.if_(alt.datum.x <= -thresh_selector.threshold, alt.datum.t_yn, 0)
)

# Control Gaussians
c_gauss_base = base.encode(
    x = alt.X('x:Q', scale=alt.Scale(domain=[xmin,xmax]), title=''),
    y = alt.Y('c_y:Q', title='')
)
c_line = c_gauss_base.mark_line(color='#666666',clip=True).add_selection(
    thresh_selector, 
    t_center_selector
)
c_area_base = c_gauss_base.mark_area(color='#C84E0099',blend='multiply',clip=True)
c_areaPos = c_area_base.transform_filter( alt.datum.x >= thresh_selector.threshold )
c_areaNeg = c_area_base.transform_filter( alt.datum.x <= -thresh_selector.threshold )

# Treatment Gaussians
t_gauss_base = base.encode(
    x = alt.X('x:Q', scale=alt.Scale(domain=[xmin,xmax]), title=''),
    y = alt.Y('t_y:Q', scale=alt.Scale(domain=[0,0.45]), title='')
)
t_line = t_gauss_base.mark_line(color='#262626',clip=True)
t_area_base = t_gauss_base.mark_area(color='#33989899',blend='multiply',clip=True)
t_areaPos = t_area_base.transform_filter( alt.datum.x >= thresh_selector.threshold )
t_areaNeg = t_area_base.transform_filter( alt.datum.x <= -thresh_selector.threshold )

# Bars
cPos_bar = base.mark_bar(size=30, color='#C84E00AA').encode(
    y = alt.Y('sum(c_ynPos):Q', scale=alt.Scale(domain=(-1, 1)), title='Control pos area · p(CI)'),
    tooltip = alt.Tooltip('sum(c_ynPos):Q', format='0.4f')
).add_selection(alt.selection_single())
cNeg_bar = base.mark_bar(size=30, color='#C84E00AA').encode(
    y = alt.Y('sum(Minusc_ynNeg):Q', scale=alt.Scale(domain=(-1,1)), title='Control neg area · p(CW)'),
    tooltip = alt.Tooltip('sum(Minusc_ynNeg):Q', format='0.4f')
).add_selection(alt.selection_single()).transform_calculate(Minusc_ynNeg = -1*alt.datum.c_ynNeg)
tPos_bar = base.mark_bar(size=30, color='#339898BB').encode(
    y = alt.Y('sum(t_ynPos):Q', scale=alt.Scale(domain=(-1, 1)), title='Treatment pos area · p(TI)'),
    tooltip = alt.Tooltip('sum(t_ynPos):Q', format='0.4f')
).add_selection(alt.selection_single())
tNeg_bar = base.mark_bar(size=30, color='#339898BB').encode(
    y = alt.Y('sum(Minust_ynNeg):Q', scale=alt.Scale(domain=(-1,1)), title='Treatment neg area · p(TW)'),
    tooltip = alt.Tooltip('sum(Minust_ynNeg):Q', format='0.4f')
).add_selection(alt.selection_single()).transform_calculate(Minust_ynNeg = -1*alt.datum.t_ynNeg)

calc_bar = base.mark_bar(size=30, color='#993399BB').encode(
    y = alt.Y('pB:Q', scale=alt.Scale(domain=(-1, 1)), title='Net benefit · p(B)'),
    tooltip = alt.Tooltip('pB:Q', format='0.4f')
).transform_aggregate(
    pTI = 'sum(t_ynPos)',
    pTW = 'sum(t_ynNeg)',
    pCI = 'sum(c_ynPos)',
    pCW = 'sum(c_ynNeg)'
).transform_calculate(
    pB = '(datum.pTI-datum.pTW) - (datum.pCI-datum.pCW) + datum.pTW*datum.pCI - datum.pTI*datum.pCW'
).add_selection(alt.selection_single())

fancy_combo = (c_areaPos + c_areaNeg + t_areaPos + t_areaNeg + c_line + t_line).properties(height = 200, width=350) | \
        cPos_bar.properties(height = 200, width=50) | \
        cNeg_bar.properties(height = 200, width=50) | \
        tPos_bar.properties(height = 200, width=50) | \
        tNeg_bar.properties(height = 200, width=50) | \
        calc_bar.properties(height = 200, width=50)

fancy_combo

alt.HConcatChart(...)

In [43]:
fancy_combo.save('net_benefit.html')